# RNN에서 Seq2Seq로

In [4]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras. layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout 
from tensorflow.keras.callbacks import EarlyStopping
# 데이터 생성 함수
def generate_data (num_samples) :
    # X = np.random.randint(0, 10, (num_samples, 5))
    X = np.random.randint(0, 10, (num_samples, 10))
    y = np.flip(X, axis=1)
    return X, y
# 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)
# 데이터 형태 변환 (LSTM을 사용하기 위해 3D 텐서로 변환)
X = X. reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))
# 모델 생성
model = Sequential()
# model.add(LSTM(128, input_shape=(5, 1)))
model.add(LSTM(128, input_shape=(10, 1)))
model.add(Dropout (0.2))
# model.add(RepeatVector(5))
model.add(RepeatVector(10))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(10, activation='softmax')))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# EarLyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# 모델 학습
model.fit(X, y, epochs=50, validation_split=0.2, callbacks=[early_stopping])
# 모델 검증 함수
def predict_reverse(input_sequence):
    # input_sequence = np.array(input_sequence).reshape((1, 5, 1))
    input_sequence = np.array(input_sequence).reshape((1, 10, 1))
    predicted_sequence = model. predict (input_sequence)
    # return np.argmax(predicted_sequence, axis=-1). reshape((5,))
    return np.argmax(predicted_sequence, axis=-1). reshape((10,))
# 입력 반복 루프


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.1448 - loss: 2.2343 - val_accuracy: 0.2113 - val_loss: 2.0399
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.2288 - loss: 1.9946 - val_accuracy: 0.2891 - val_loss: 1.8182
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.2850 - loss: 1.7984 - val_accuracy: 0.3605 - val_loss: 1.6024
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.3576 - loss: 1.5867 - val_accuracy: 0.5040 - val_loss: 1.2829
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.4547 - loss: 1.3426 - val_accuracy: 0.6326 - val_loss: 1.0358
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5442 - loss: 1.1156 - val_accuracy: 0.7404 - val_loss: 0.8512
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6100 - loss: 0.9695 - val_accuracy: 0.8151 - val_loss: 0.6917
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6564 - loss: 0.8608 - val_accuracy: 

In [6]:
while True:
    # user_input = input ("입력 (쉼표로 구분된 5개의 숫자, 종료하려면 'quit' 입력): ")    
    user_input = input ("입력 (쉼표로 구분된 10개의 숫자, 종료하려면 'quit' 입력): ")

    if user_input. lower () =="quit":
        break   
    try:
        input_sequence = list(map(int, user_input.split(',')))
        # if len(input_sequence) != 5:
        if len(input_sequence) != 10:
            # print("5개의 숫자를 입력해 주세요.")
            print("10개의 숫자를 입력해 주세요.")
            continue 
        predicted_output = predict_reverse(input_sequence)
        print(f"입력: {input_sequence}")
        print(f"예측된 출력: {predicted_output}")
    except ValueError:
        print("유효한 숫자를 입력해 주세요.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
입력: [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
예측된 출력: [0 9 8 7 6 5 4 3 2 1]


In [8]:
# 20개의 data를 만들어 검증 # 검증용 데이터 생성
test_X, test_y = generate_data(20)
# 모델 예측 및 정확도 계산
correct_predictions = 0
total_predictions = len(test_X)
for i in range(total_predictions):
    input_sequence = test_X[i]
    # true_output = test_y[i].reshape(5,)
    true_output = test_y[i].reshape(10,)
    predicted_output = predict_reverse(input_sequence)
    if np.array_equal(predicted_output, true_output):
        correct_predictions += 1
    # print(f"입력: {input_sequence.reshape(5,)}")
    print(f"입력: {input_sequence.reshape(10,)}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {true_output}")
    print(f"정확 여부: {'맞음' if np.array_equal(predicted_output, true_output) else '틀림'}\n")
accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
입력: [7 8 6 4 2 8 6 0 5 9]
예측된 출력: [9 5 0 6 8 2 4 6 8 7]
실제 출력: [9 5 0 6 8 2 4 6 8 7]
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
입력: [6 4 3 7 7 1 2 9 0 6]
예측된 출력: [6 0 9 2 1 7 7 3 4 6]
실제 출력: [6 0 9 2 1 7 7 3 4 6]
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: [2 9 3 9 9 9 4 4 8 4]
예측된 출력: [4 8 4 4 9 9 9 3 9 2]
실제 출력: [4 8 4 4 9 9 9 3 9 2]
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: [5 0 9 4 2 1 3 6 4 7]
예측된 출력: [7 4 6 3 1 2 5 9 0 5]
실제 출력: [7 4 6 3 1 2 4 9 0 5]
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: [8 9 8 4 6 9 4 7 9 3]
예측된 출력: [3 9 7 4 9 6 4 8 9 8]
실제 출력: [3 9 7 4 9 6 4 8 9 8]
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
입력: [4 9 5 8 9 6 9 2 2 5]
예측된 출력: [5 2 2 9 6 9 8 5 9 4]
실제 출력: [5 2 2 9 6 9 8 5 9 4]
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: [8 1 7 1 5 1 1 0 0 8]
예측된 출력: [8 0 0 1 1 5 1 7 1 8]
실제 출력: [8 0 0 1 1 5 1 7 1 8]
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: [7 1 2 9 8 2 1 6 6 6]
예측된

# RNN에서 Seq2Seq로 - 인코더, 디코더 구분 구현

In [11]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras. layers import Dense, LSTM, RepeatVector, Input, Dropout 
from tensorflow.keras.callbacks import EarlyStopping
# 데이터 생성 함수
def generate_data(num_samples) :
    X = np.random.randint(0, 10, (num_samples, 5))
    # X = np.random.randint(0, 10, (num_samples, 10))
    y = np.flip(X, axis=1)
    return X, y
# 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)
# 데이터 형태 변환 (LSTM을 사용하기 위해 3D 텐서로 변환)
X = X. reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

# 인코더 정의
encoder_inputs = Input(shape=(5, 1))
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder (encoder_inputs)
encoder_states = [state_h, state_c]
# 디코더 정의
decoder_inputs = Input(shape=(5, 1))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(10, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [12]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 입력 데이터와 타겟 데이터를 동일하게 맞춤
decoder_input_data = np.zeros_like(X)

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training
model.fit([X, decoder_input_data], y, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# Validation function
def predict_reverse(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, 5, 1))
    # input_sequence = np.array(input_sequence).reshape((1, 10, 1))
    decoder_input = np.zeros((1, 5, 1))
    predicted_sequence = model.predict ([input_sequence, decoder_input])
    return np.argmax(predicted_sequence, axis=-1). reshape((5,))
    # return np.argmax(predicted_sequence, axis=-1). reshape((10,))
    

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.2133 - loss: 2.0189 - val_accuracy: 0.7829 - val_loss: 0.8633
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8823 - loss: 0.6766 - val_accuracy: 0.9938 - val_loss: 0.2826
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9924 - loss: 0.2271 - val_accuracy: 0.9993 - val_loss: 0.0908
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9907 - loss: 0.0972 - val_accuracy: 1.0000 - val_loss: 0.0447
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 0.0369 - val_accuracy: 1.0000 - val_loss: 0.0222
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 0.0192 - val_accuracy: 1.0000 - val_loss: 0.0128
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9949 - loss: 0.0283 - val_accuracy: 0.9997 - val_loss: 0.0174
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9998 - loss: 0.0121 - val_accuracy: 1.

# 문자로 변환

In [94]:
import numpy as np 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.layers import Bidirectional 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.sequence import pad_sequences

cal_len = 15
# 문자와 인덱스 매핑
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for idx, char in enumerate(chars)}
# 데이터 생성 함수
def generate_data(num_samples):
    X = []
    y = []
    for _ in range(num_samples):
        sequence = np.random.choice(list(chars), cal_len)
        X.append([char_to_index[char] for char in sequence]) 
        y.append([char_to_index[char] for char in sequence[::-1]])
    return np.array(X), np.array(y)

# 학습 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)

# Reshape data
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

In [95]:
# 인코더 정의
encoder_inputs = Input(shape=(cal_len, 1))
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# 디코더 정의
decoder_inputs = Input(shape=(cal_len, 1))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(52, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [96]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 입력 데이터와 타겟 데이터를 동일하게 맞춤
decoder_input_data = np.zeros_like(X)

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X, decoder_input_data], y, epochs=50, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0318 - loss: 3.8546 - val_accuracy: 0.0680 - val_loss: 3.4607
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.0797 - loss: 3.3433 - val_accuracy: 0.1063 - val_loss: 3.0500
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1232 - loss: 2.9471 - val_accuracy: 0.1398 - val_loss: 2.7803
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1581 - loss: 2.6722 - val_accuracy: 0.1773 - val_loss: 2.5044
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1913 - loss: 2.4523 - val_accuracy: 0.2048 - val_loss: 2.3463
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.2218 - loss: 2.2846 - val_accuracy: 0.2330 - val_loss: 2.1898
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.2482 - loss: 2.1499 - val_accuracy: 0.2471 - val_loss: 2.1063
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2742 - loss: 2.0402 - val_accuracy: 0.

In [97]:
def predict_reverse(input_sequence):
    input_sequence = np.array([char_to_index[char] for char in input_sequence]).reshape((1, cal_len, 1))    
    decoder_input = np.zeros((1, cal_len, 1))
    predicted_sequence = model.predict ([input_sequence, decoder_input])
    predicted_indices = np.argmax(predicted_sequence, axis=-1). reshape((cal_len,))
    return "".join([index_to_char[idx] for idx in predicted_indices])
    # return np.argmax(predicted_sequence, axis=-1). reshape((10,))

In [98]:
#검증용 문자열 리스트 (5글자짜리 30개)
test_strings = ["".join(np.random.choice(list(chars), cal_len)) for _ in range(30)]
# 실제 거꾸로 된 문자열 리스트
expected_outputs = [s[::-1] for s in test_strings]
# 모델 검증 및 정확도 계산
correct_predictions = 0
total_predictions = len(test_strings)

In [99]:
for i, test_string in enumerate(test_strings):
    predicted_output = predict_reverse(test_string)
    is_correct = predicted_output == expected_outputs[i]
    
    if is_correct:
        correct_predictions += 1
        
    print(f"입력: {test_string}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {expected_outputs[i]}")
    print(f"정확 여부: {'맞음' if is_correct else '틀림'}\n")

accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
입력: OUldcymnQGPWWkD
예측된 출력: DkWWPGPomxddlUO
실제 출력: DkWWPGQnmycdlUO
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
입력: CNGSmMcnXynarVc
예측된 출력: cVqanzXncLlSGOC
실제 출력: cVranyXncMmSGNC
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: NcnksmwjKOKHEYs
예측된 출력: sYEHKOKgwmtlncN
실제 출력: sYEHKOKjwmskncN
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: RfPIetUSTjHcUhB
예측된 출력: BiUcIkSSTsgIPfQ
실제 출력: BhUcHjTSUteIPfR
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: jwFfYEVwAlXilyS
예측된 출력: SyljXlAxVEYgFwi
실제 출력: SyliXlAwVEYfFwj
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: puvIekrkuriIQiA
예측된 출력: AiRIirtkskfIuvo
실제 출력: AiQIirukrkeIvup
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: SUEGiEjuSYryJPb
예측된 출력: bPJysZTtjEjGEUS
실제 출력: bPJyrYSujEiGEUS
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: bIwLVGpQIfBiZLB
예측된 출력: BLajBeIPqGVLwIb
실제 출력: BLZiBfIQpGVLwIb
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: oWZvYRfhyhjFtiE
예측된 출력: EisFjhyifRXv